# Example of Pressure-Deflection Diagram

In [1]:
from pygasflow.shockwave import PressureDeflectionLocus
from pygasflow.interactive import PressureDeflectionDiagram
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

## Simple shock reflection process

This is a reproduction of figure 4.22 from *Modern Compressible Flow: with historical perspective, J.D. Anderson*.

In [2]:
# Known data
gamma = 1.4
M1 = 2.8
theta1 = 16

l1 = PressureDeflectionLocus(M=M1, gamma=gamma, label="1")
l2 = l1.new_locus_from_shockwave(theta1, label="2")

d = PressureDeflectionDiagram()
d.add_locus(l1)
d.add_locus(l2)
l, arrows = d.add_path((l1, theta1), (l2, 0))
d.add_state(0, l2.eval_at_theta(0)[0], "3")
d.move_legend_outside()
d.y_range = (0, 14)
d

## Intersection of shocks of opposite families

Here I reproduce figure 4.24 from *Modern Compressible Flow: with historical perspective, J.D. Anderson* using data from [MAE 5420 - Compressible Fluid Flow - Section 9.1](http://mae-nas.eng.usu.edu/MAE_5420_Web/section9/section9.1.pdf).

The problem is depicted in this picture:

![test](figure_4_23.png)

In [3]:
M1 = 3
theta_2 = 20
theta_3 = -15

l1 = PressureDeflectionLocus(M=M1, label="1")
l2 = l1.new_locus_from_shockwave(theta_2, label="2")
l3 = l1.new_locus_from_shockwave(theta_3, label="3")

phi, p4_p1 = l2.intersection(l3)
print("Intersection between locus M2 and locus M3 happens at:")
print("Deflection Angle [deg]:", phi)
print("Pressure ratio to freestream:", p4_p1)
print()
print("From geometry:")
print("theta_4 [deg] =", phi - l3.theta_origin)
print("theta_4' [deg] =", phi - l2.theta_origin)

d = PressureDeflectionDiagram()
d.add_locus(l1)
d.add_locus(l2)
d.add_locus(l3)
d.add_path((l1, theta_2), (l2, phi))
d.add_path((l1, theta_3), (l3, phi))
d.add_state(
    phi, p4_p1, "4=4'", 
    background_fill_color="white",
    background_fill_alpha=0.8)
d.move_legend_outside()
d.y_range = (0, 18)
d

Intersection between locus M2 and locus M3 happens at:
Deflection Angle [deg]: 4.795958931693682
Pressure ratio to freestream: 8.352551913417367

From geometry:
theta_4 [deg] = 19.795958931693683
theta_4' [deg] = -15.204041068306317
